# Final Problem

Final exercise for Machine Learning and Deep Learning ABK

With many thanks to Lisa Benato

# Quick Links:
    
Input files: https://desycloud.desy.de/index.php/s/3wJef7MdfekewTD
        
Leaderboard: https://github.com/gkasieczka/Challenge
        
File Submission: https://desycloud.desy.de/index.php/s/TdxsKpsKGNzEngE

Program Submission: https://desycloud.desy.de/index.php/s/WaScZPGQ9Ypj9Yj

(always upload both!)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import pandas

### The Standard Model and the top quark

<br>
<img src="img/SM.png" width="400" >

The **Standard Model** of elementary particles represents our knowledge of the microscopic world. It describes the matter constituents (quarks and leptons) and their interactions (mediated by bosons), that are the electomagnetic, the weak and the strong interactions.

Among all these particles, the **top quark** still represents a very peculiar case. It is the heaviest known elementary particle (mass of 172.5 GeV) and it has a very short lifetime ($10^{-25}$ seconds): this means we can only see its decay products. It has been discovered in 1995 by CDF and D0 experiments at Tevatron (Fermilab, Chicago). The top quark is considered a key particle to searches for new physics beyond the Standard Model and to precision measurements.

The ideal tool for measuring the top quark properties is a particle collider. The **Large Hadron Collider** (LHC), situated nearby Geneva, between France and Switzerland, is the largest proton-proton collider ever built on Earth. It consists into a 27 km circumference ring, where proton beams are smashed at a centre-of-mass energy of 13 TeV (99.999999% of speed of light). At the LHC, 40 Million collisions / second occurs, providing an enormous amount of data. Thanks to these data, **ATLAS** and **CMS** experiments discovered the missing piece of the Standard Model, the Higgs boson, in 2012.

During a collision, the energy is so high that protons are "broken" into their fundamental components, i.e. **quarks** and **gluons**, that can interact together, producing particles that we don't observe in our everyday life, such as the top quark. The production of a top quark is, by the way, a relatively "rare" phenomenon, since there are other physical processes that occur way more often, such those initiated by strong interaction, producing lighter quarks (such as up, down, strange quarks). In high energy physics, we speak about the **cross-section** of a process. We say that, the top quark production has a smaller cross-section than the production of light quarks.

The experimental consequence is that distinguishing the decay products of a top quark from a light quark can be extremely difficult, given that the latter phenomenon has a way larger probability to happen.

### Experimental signature of top quark in a particle detector

Let's first understand what are the experimental signatures and how our detectors work. This is a sketch of the CMS experiment.

<br>
<img src="img/EPS_CMS_Slice.png" width="1000" >

A collider detector is organized in layers: each layer is able to distinguish and measure different particles and their properties. For example, the silicon tracker detects each particle that is charged. The electromagnetic calorimeter detects photons and electrons. The hadronic calorimeter detects hadrons (such as protons and neutrons). The muon chambers detect muons (that have a long lifetime and travel through the inner layers).

Our physics problem consists into detecting the so-called "hadronic decay" of a top quark. The decay chain is sketched here: the top quark decays into a bottom quark and into a $W$ boson, that in turn decays into light quarks (in the picture, up and down quarks).

<br>
<img src="img/top.png" width="500" >

Our background is, instead, represented by light quark (or quarks) produced by the strong interaction (in jargon, QCD). Here we have a sketch of one possible background event.

<br>
<img src="img/QCD.png" width="200" >

#### Jets

Without going into the theoretical details, the nature of particles experiencing the strong interaction (like quarks) is such that they cannot travel free, but they are forced to be "confined" into hadrons. One hadron can be seen as a "combination" of quarks. Let's think about the electromagnetic interaction: a positive charge and a negative charge are attracted to each other, and they will tend to form a state that is neutral under the electromagnetic interaction. Analogously, quarks try to combine together, forming a bond state that is neutral under the strong interaction. This process is called **hadronization**, and it has a very important consequence. Quarks won't appear as single isolated particles in a detetctor, but rather as **jets** of particles.

There are many different algorithms that are able to reconstruct quarks (and gluons) as jets (i.e., anti-$k_T$ algorithm https://arxiv.org/abs/0802.1189). They basically loop over the shower of particles produced by the hadronization, trying to cluster them together as one single entity. The algorithms are designed such in a way that the momentum of the clustered jet is proportional to the initial energy of the quark. A sketch giving an intuitive idea of a jet is displayed here (Klaus Rabbertz, KIT):

<br>
<img src="img/Rabbertz_from_quark_to_rec_jet.png" width="500" >

#### Jets substructure

Many physically motivated approaches have been used in the past to distinguish a jet initiated by a top quark from jets due to QCD. One remarkable property is the so-called **jet substructure**. The idea is to try to distinguish how many "sub-jets" are included in a jet. Out of our sketches presented before, since the top quark decays into three separated quarks, we would expect it to show a three-pronged sub-structure. QCD, on the other hand, is mainly due to single quark/gluon radiation, hence it shows a one-pronged sub-structure. One largely used approach to study the jet substructure is the so called *n-subjettiness* (https://arxiv.org/abs/1011.2268).

# Four Vector Data
Our problem is then classifying a jet produced by a top quark (that we will call signal event) from a light quark jet due to strong interaction (background event).

In this exercise, a quite large dataset is provided. Each event is a Monte Carlo simulation of a jet candidate, that can be signal or background. An integer (0 or 1) will label each jet as background or signal. For each jet, the 4-momenta of its constituents (i.e., the particles that are clustered by the jet algorithm) are stored. One can build neural network architectures that are able to understand the nature of a jet only by looking at its constituents.

The very first approach we are going to have in this exercise is trying a classical artificial neural network approach, that is, Fully Connected Neural Networks. They are very generic structures, that can be applied in many different classification problems, but that sometimes provides many weights and that can be quite inefficient. 

Some bibliography about Fully Connected neural network approaches for top quark tagging can be found here:

https://arxiv.org/abs/1704.02124
https://arxiv.org/abs/1501.05968

First, let's have a look at the shape of the input training data. They are stored as `pandas` **`DataFrame`** HDF5 data, in the format of `PyTables`. 

We have 100k rows, i.e. 100k different events, and 806 columns (whose meaning will be explained later).

One useful hint: keep the last events of the training set as test samples to evaluate performances. In order to do so, you can use **`stop`** and **`start`** options in **`select`**:

In [ ]:
# Read the first 20k events
# They are quite few, to be increased in later steps
store_train = pandas.HDFStore("train_100k.h5")
df_train = store_train.select("table",stop=20000)
print(df_train.shape)

In [ ]:
# Keep the last 2k events as test sample
# Suggested in next steps: take up to 100k events for testing
# But remember: do not use the same events for both training and testing!
df_test = store_train.select("table",start=100000-2000)
print(df_test.shape)

Let's print out the first three rows of this dataset:

In [ ]:
df_test.iloc[0:3]

The first 800 columns contain the 4-momenta, i.e. energy, $p_x$, $p_y$ and $p_z$, of the first 200 jet constituents. If the constituents are less than 200, the values of the 4-momenta are filled with zeros. The `is_signal_new` column contains the truth information, i.e. it tells if that particular event is signal (`is_signal_new`=1) or background (`is_signal_new`=0). We can disregard the columns starting with "truth" and "ttv".

Let's first focus on the first 20 particle constituents and let's define a list of strings that will be very convenient when loading the pandas **`DataFrame`**.

In [ ]:
# four-momenta of leading 20 particles
cols = [c.format(i) for i in range(20) for c in ["E_{0}",  "PX_{0}",  "PY_{0}",  "PZ_{0}"]]

vectors_test = df_test[cols].to_numpy()
vectors_test_labels = df_test["is_signal_new"].to_numpy()

vectors_train = df_train[cols].to_numpy()
vectors_train_labels = df_train["is_signal_new"].to_numpy()

# Image Data


Above, we have described a very easy approach to tackle an high energy physics problem. We can use different approach, that represents the state-of-the-art of a more general problem, that is, image recognition.

We have described what is a jet and how we can reconstruct it, by clustering together the particles produced in the hadronization process of a quark (or a gluon). Let's think about the CMS detector: its shape is basically that of a cylinder. The cylindrical surface of the detector can be unrolled along the radial and the longitudinal coordinates, This surface, that will be a rectangle, can then be divided into "pixels". The particle energy deposits can be converted into "colour intensities" within each pixel. The more dense and the more energetic the particles, the more color density in one particular pixel.

The idea behind jet images is a specific application of the previously described approach: the energy deposits of the jets constituents are transformed into "intensities" of a 2D black and white image. Image recognition algorithms can be therefore applied to a high energy physics problem.

<br>
<img src="img/images_jets.png" width="800" >

https://arxiv.org/abs/1612.01551

The images that we are going to use require some pre-processing, such as being properly normalized and oriented. The details are included in the following papers:
* https://arxiv.org/abs/1701.08784
* https://arxiv.org/abs/1803.00107
* https://arxiv.org/abs/1407.5675
* https://arxiv.org/abs/1511.05190

This time, we have converted the information of the energy of the jet constituents into an image. Each image has 40x40 pixels = 1600 pixels. Each column represents the *colour* intensity in each pixel.

Let's define a function, `to_image`, that rewrites these columns as a 40x40 numpy matrix, with one additional index, that represents the colours of the image (see later boxes).

In [ ]:
# 1 image has 40x40 pixels = 1600 pixels
pixels = ["img_{0}".format(i) for i in range(1600)]

def to_image(df):
    return  np.expand_dims(np.expand_dims(df[pixels], axis=-1).reshape(-1,40,40), axis=1)

Let's now read only the first 20k events that will be used for training (you can increase this number later):


In [ ]:
# Read the first 20k events
store_train_images = pandas.HDFStore("train_img_100k.h5")
df_train_images = store_train_images.select("table",stop=20000)

images_train = to_image(df_train_images)
images_train_labels = df_train_images["is_signal_new"].to_numpy()

One useful hint: keep the last events of the training set as test samples to evaluate performances. In order to do so, you can use **`stop`** and **`start`** options in **`select`**. Let's prepare a test sample.

In [ ]:
# Keep the last 2k events as test sample
df_test_images = store_train_images.select("table",start= 100000 - 2000)
images_test = to_image(df_test_images)
images_test_labels = df_test_images["is_signal_new"].to_numpy()

Let's have a look at the shape of the input training data:


In [ ]:
images_train.shape

As stated before, this time we have a more complex input shape, a multidimensional matrix with four indices:

* The first index is the number of training events
* The second index labels the colours of the images
* The third index labels the pixels of the images (40) along the horizontal axis
* The fourth index labels the pixels of the images (40) along the vertical axis


In modern image recognition architectures, it is very common to decompose an image into three colour layers: red, green and blue. Our jet images have only one colour (they are black and white), since we are converting the momenta of the particles hitting that particular region of the detector as a color intensity.

In [ ]:
# Save all signal events in one DataFrame
df_signal = df_train_images[df_train_images["is_signal_new"]==1]
# Save all background events in one DataFrame
df_background = df_train_images[df_train_images["is_signal_new"]==0]

#Perform an average of signal and background DataFrame
df_signal_sum = df_signal.sum(axis = 0, skipna = True)/len(df_signal.index)
df_background_sum = df_background.sum(axis = 0, skipna = True)/len(df_background.index)

#Define an easier function to convert the averaged images into 40x40 matrices
def to_image_plot(df):
    return  df[pixels].values.reshape(40,40)

image_signal = to_image_plot(df_signal_sum)
image_background = to_image_plot(df_background_sum)

Let's plot signal and background images.

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(15,15))
plt.rcParams.update({'font.size': 15}) #Larger font size

axs[0].imshow(image_signal,cmap="Reds")
axs[0].set_title('Signal')

axs[1].imshow(image_background,cmap="Blues")
axs[1].set_title('Background')

for ax in axs.flat:
    ax.set(xlabel='$\eta$', ylabel='$\phi$')
 
plt.show()

# Networks and training code

In [ ]:
# Fully Connected Architecture
class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        
        self.fc1 = nn.Linear(80, 2)        

    def forward(self, x):
        
        x = self.fc1(x)
        
        return x    

# Convolutional Architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 4, 3, padding = True)        
        self.fc1 = nn.Linear(40 * 40 * 4, 2)
        
    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        
        x = x.view(-1, 40*40*4)
        x = F.relu(self.fc1(x))
        
        return x
  

# Which device to use for NN calculations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

# Loss function
criterion = nn.CrossEntropyLoss()

# how many epochs to train for
n_epochs = 20

# how many examples / batch
batch_size = 100

# !!!! Which data and architecture to use
do_image = True

# Keep track of the accuracies
train_accs = []
test_accs = []

In [ ]:
if do_image:
    print("Training Images")
    model = CNN().to(device)
    X_train = images_train
    y_train = images_train_labels
    
    X_test = images_test
    y_test = images_test_labels
else:
    print("Training Vectors")
    model = FCN().to(device)
    X_train = vectors_train
    y_train = vectors_train_labels
    
    X_test = vectors_test    
    y_test = vectors_test_labels
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Create network object
train_examples = X_train.shape[0]
n_batches = int(train_examples/batch_size)

# Loop over the epochs
for ep in range(n_epochs):

    # reorder the training events for each epoch
    idx = np.arange(X_train.shape[0])
    np.random.shuffle(idx)
    
    X_train = X_train[idx]
    y_train = y_train[idx]
    
    # Each epoch is a complete loop over the training data
    for i in range(n_batches):
        
        # Reset gradient
        optimizer.zero_grad()
        
        i_start = i*batch_size
        i_stop  = (i+1)*batch_size
        
        # Convert x and y to proper objects for PyTorch
        x = torch.tensor(X_train[i_start:i_stop],dtype=torch.float)
        y = torch.tensor(y_train[i_start:i_stop],dtype=torch.long)

        # Apply the network 
        net_out = model(x)
        
        # Calculate the loss function
        loss = criterion(net_out,y)
                
        # Calculate the gradients
        loss.backward()
        
        # Update the weights
        optimizer.step()
    # end of loop over batches
        
    # Calculate predictions on training and testing data
    y_pred_train = nn.Softmax(dim=1)(model(torch.tensor(X_train,dtype=torch.float))).detach().numpy()
    y_pred = nn.Softmax(dim=1)(model(torch.tensor(X_test,dtype=torch.float))).detach().numpy()

    # Calculate accuracy on training and testing data
    train_acc = sum(y_train == np.argmax(y_pred_train,axis=1)) / y_train.shape[0]
    test_acc = sum(y_test == np.argmax(y_pred,axis=1)) / y_test.shape[0]
    
    # print some information
    print("Epoch:",ep, "Train Accuracy:", train_acc,  "Test Accuracy:", test_acc)
    
    # and store the accuracy for later use
    train_accs.append(train_acc)
    test_accs.append(test_acc)
# end of loop over epochs
    
# Prepare and show accuracy over time
plt.axis('on')
plt.plot(train_accs,label="train")
plt.plot(test_accs,label="test")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

print("Best test accuracy:",max(test_accs))

In [ ]:
### Evaluate performance on independent sample
# DO NOT CHANGE BELOW!
from IPython.display import FileLink, FileLinks

if do_image:
    print("Loading input for images")
    store_test_challenge = pandas.HDFStore("test_without_truth_img_10k.h5")
    df_test_challenge = store_test_challenge.select("table")
    x = torch.tensor(to_image(df_test_challenge),dtype=torch.float)    
else:
    print("Loading input for vectors")
    store_test_challenge = pandas.HDFStore("test_without_truth_10k.h5")
    df_test_challenge = store_test_challenge.select("table")
    x = torch.tensor(df_test_challenge[cols].to_numpy(),dtype=torch.float)
    
print("Running on full test sample. This may take a moment.")
out = model(x)
ret = out.detach().numpy()

np.save("result.npy",ret[:,1])
!zip result.zip result.npy
print("Done. Click below  to download result. Rename and upload to submission page. Also remember to upload the code!")
FileLink('result.zip')